In [2]:

# Final Project
# Student Name: Rawan Elshahat
# Project Title: Library Management System using Object-Oriented Programming (OOP)

# Description: This project is a simple system to organize a library using OOP concepts in Python.
#It manages books, users, borrowing and returning operations with clear menus and data validation.
#The system also includes logging, history tracking, and saving data to files for future use.


# Selected System:  Library Management
# OOP Features Implemented:

#Classes and Objects:Used classes for `Book`, `User`, `Admin`, `Member`, and `Library`.
#Encapsulation: Implemented private attribute `__quantity` in `Book` with getter and setter methods.
#Inheritance: `Admin` and `Member` classes inherit from `User`.
#Polymorphism (Method Overriding):** `borrow_book()` method is overridden in `Admin` and `Member` to provide different behavior.
#Modularity: Each class has its own methods for handling its own data and responsibilities.




import datetime
import json

In [3]:
# ==================== Book ====================
class Book:
    def __init__(self, book_id, title, author, quantity):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.__quantity = quantity  # private attribute

    # Getter
    def get_quantity(self):
        return self.__quantity

    # Setter
    def set_quantity(self, value):
        if value > 0:
            self.__quantity = value
        else:
            print("Quantity cannot be -ve")

    def display_book_info(self):
        print(f"ID: {self.book_id}, Title: {self.title}, Author: {self.author}, Available Quantity: {self.__quantity}")

    def check_availability(self):
        return self.__quantity > 0

    def update_quantity(self, quantity):
        self.set_quantity(self.__quantity + quantity)



In [4]:
# ==================== User (Parent) ====================
class User:
    def __init__(self, user_id, name):
        self.user_id = user_id
        self.name = name
        self.borrowed_books = []  # List to store borrowed books

    def borrow_book(self, book):
        if book.check_availability():
            self.borrowed_books.append(book)
            book.update_quantity(-1)
            print(f"{self.name} has borrowed '{book.title}'")
            library.log_operation(f"{self.name} borrowed '{book.title}'")
        else:
            print(f"Sorry, '{book.title}' is not available for borrowing.")

    def return_book(self, book):
        if book in self.borrowed_books:
            self.borrowed_books.remove(book)
            book.update_quantity(1)
            print(f"{self.name} has returned '{book.title}'")
            library.log_operation(f"{self.name} returned '{book.title}'")
        else:
            print(f"{self.name} does not have '{book.title}' borrowed.")

    def view_borrowed_books(self):
        if self.borrowed_books:
            print(f"{self.name}'s Borrowed Books:")
            for book in self.borrowed_books:
                print(f" - {book.title}")
        else:
            print(f"{self.name} has not borrowed any books.")


# ==================== Admin (Child) ====================
class Admin(User):
    # override
    def borrow_book(self, book):
        print("(Admin privilege)")
        super().borrow_book(book)


# ==================== Member (Child) ====================
class Member(User):
    # override
    def borrow_book(self, book):
        if len(self.borrowed_books) >= 3:
            print(f"{self.name} cannot borrow more than 3 books.")
        else:
            super().borrow_book(book)


In [5]:


# ==================== Library ====================
class Library:
    def __init__(self):
        self.books = []
        self.users = []
        self.history = []  # log of operations

    def log_operation(self, message):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        entry = f"[{timestamp}] {message}"
        self.history.append(entry)
        print(entry)

    def add_book(self, book):
        self.books.append(book)
        self.log_operation(f"Book added: {book.title}")

    def register_user(self, user):
        if any(curr_user.user_id == user.user_id for curr_user in self.users):
            print("User with the same ID already exists! Try another ID.")
            return
        self.users.append(user)
        self.log_operation(f"New user {user.name} added successfully!")

    def search_book_by_title(self, title):
        found_books = [book for book in self.books if title.lower() in book.title.lower()]
        return found_books

    def list_books(self):
        if self.books:
            print("Books in the Library:")
            for book in self.books:
                book.display_book_info()
        else:
            print("No books available in the library.")

    def add_new_book(self):
        try:
            book_id = int(input("Enter book ID: "))
            if any(book.book_id == book_id for book in self.books):
                print("Book with the same ID already exists, please try another ID.")
                return
            title = input("Enter book title: ")
            author = input("Enter author name: ")
            quantity = int(input("Enter the quantity of books: "))
            new_book = Book(book_id, title, author, quantity)
            self.add_book(new_book)
            print("Book added successfully!")
            new_book.display_book_info()
        except ValueError:
            print("Invalid input, please try again.")

    def save_data(self):
        data = {
            "books": [
                {"id": b.book_id, "title": b.title, "author": b.author, "quantity": b.get_quantity()}
                for b in self.books
            ],
            "users": [
                {"id": u.user_id, "name": u.name, "role": u.__class__.__name__}
                for u in self.users
            ]
        }
        with open("library_data.json", "w") as f:
            json.dump(data, f, indent=4)
        self.log_operation("Library data saved to library_data.json")

    def load_data(self):
        try:
            with open("library_data.json", "r") as f:
                data = json.load(f)

            self.books = []
            for b in data.get("books", []):
                self.books.append(Book(b["id"], b["title"], b["author"], b["quantity"]))

            self.users = []
            for u in data.get("users", []):
                if u["role"] == "Admin":
                    self.users.append(Admin(u["id"], u["name"]))
                else:
                    self.users.append(Member(u["id"], u["name"]))

            self.log_operation("Library data loaded successfully")
        except FileNotFoundError:
            self.log_operation("No previous data found. Starting fresh.")

In [6]:

# ==================== Main ====================
def main():
    global library
    library = Library()
    library.load_data()

    # Add sample data
    if not library.books:
        library.add_book(Book(1, "Harry Potter", "J.K. Rowling", 1))
        library.add_book(Book(2, "Pride and Prejudice", "Jane Austen", 2))
        library.add_book(Book(3, "1984", "George Orwell", 3))

    if not library.users:
        library.register_user(Admin(1, "Admin_1"))
        library.register_user(Admin(2, "Admin_2"))
        library.register_user(Admin(3, "Admin_3"))


        library.register_user(Member(100, "Member_1"))
        library.register_user(Member(101, "Member_2"))
        library.register_user(Member(102, "Member_3"))
        library.register_user(Member(103, "Member_4"))
        library.register_user(Member(104, "Member_5"))


    while True:
        print("\nWelcome to the Library Management System")
        print("1. View all books")
        print("2. Add books")
        print("3. Search for a book by title")
        print("4. Borrow a book")
        print("5. Return a book")
        print("6. View borrowed books")
        print("7. Add new User")
        print("8. Show history log")
        print("9. Save and Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            library.list_books()

        elif choice == '2':
            library.add_new_book()

        elif choice == '3':
            title = input("Enter the book title to search: ")
            found_books = library.search_book_by_title(title)
            if found_books:
                for book in found_books:
                    book.display_book_info()
            else:
                print(f"No books found with the title '{title}'.")

        elif choice == '4':
            try:
                user_id = int(input("Enter your user ID: "))
                book_id = int(input("Enter the book ID to borrow: "))
                user = next((u for u in library.users if u.user_id == user_id), None)
                book = next((b for b in library.books if b.book_id == book_id), None)
                if user and book:
                    user.borrow_book(book)
                else:
                    print("Invalid user or book ID.")
            except ValueError:
                print("Invalid input!")

        elif choice == '5':
            try:
                user_id = int(input("Enter your user ID: "))
                book_id = int(input("Enter the book ID to return: "))
                user = next((u for u in library.users if u.user_id == user_id), None)
                book = next((b for b in library.books if b.book_id == book_id), None)
                if user and book:
                    user.return_book(book)
                else:
                    print("Invalid user or book ID.")
            except ValueError:
                print("Invalid input!")

        elif choice == '6':
            try:
                user_id = int(input("Enter your user ID: "))
                user = next((u for u in library.users if u.user_id == user_id), None)
                if user:
                    user.view_borrowed_books()
                else:
                    print("Invalid user ID.")
            except ValueError:
                print("Invalid input!")

        elif choice == '7':
            try:
                id = int(input("Enter user id: "))
                name = input("Enter user's name: ")
                role = input("Enter role (admin/member): ").strip().lower()
                if role == "admin":
                    user = Admin(id, name)
                else:
                    user = Member(id, name)
                library.register_user(user)
            except ValueError:
                print("Invalid input!")

        elif choice == '8':
            if library.history:
                print("Operation History:")
                for log in library.history:
                    print(log)
            else:
                print("No operations recorded yet.")

        elif choice == '9':
            library.save_data()
            print("Data saved. Goodbye!")
            break

        else:
            print("Invalid choice, please try again.")


if __name__ == "__main__":
    main()


[2025-08-26 18:39:22] No previous data found. Starting fresh.
[2025-08-26 18:39:22] Book added: Harry Potter
[2025-08-26 18:39:22] Book added: Pride and Prejudice
[2025-08-26 18:39:22] Book added: 1984
[2025-08-26 18:39:22] New user Admin_1 added successfully!
[2025-08-26 18:39:22] New user Admin_2 added successfully!
[2025-08-26 18:39:22] New user Admin_3 added successfully!
[2025-08-26 18:39:22] New user Member_1 added successfully!
[2025-08-26 18:39:22] New user Member_2 added successfully!
[2025-08-26 18:39:22] New user Member_3 added successfully!
[2025-08-26 18:39:22] New user Member_4 added successfully!
[2025-08-26 18:39:22] New user Member_5 added successfully!

Welcome to the Library Management System
1. View all books
2. Add books
3. Search for a book by title
4. Borrow a book
5. Return a book
6. View borrowed books
7. Add new User
8. Show history log
9. Save and Exit
Enter your choice: 1
Books in the Library:
ID: 1, Title: Harry Potter, Author: J.K. Rowling, Available Quant